In [1]:
#===================================================================
# ElasticSearch(이하:ES) 접속 모듈 설치 및 테스트
# => ES 접속 모듈 설치 후 ES 서버에 접속하고 테스트 해보는 예제
# => ES 서버는 이미 구성되어 있다는 전제에서 접속 및 검색 테스트 해봄
#===================================================================

# elasticsearch 서버 접속 모듈 설치
# pip install elasticsearch

from elasticsearch import Elasticsearch
from elasticsearch import helpers

# elastic 서버 접속 
#es = Elasticsearch("https://192.168.0.91:9200/", verify_certs=False)
es = Elasticsearch("http://192.168.0.130:9200/")
es.info()

C:\Users\bong9\anaconda3\envs\pytorch\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'vmware',
 'cluster_name': 'mpower',
 'cluster_uuid': 'JJ1h3dNTRvSLJFW3gj5MCw',
 'version': {'number': '7.17.3',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': '5ad023604c8d7416c9eb6c0eadb62b14e766caff',
  'build_date': '2022-04-19T08:11:19.070913226Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [3]:
# 인덱스 생성
# => 인덱스는 독립된 파일 집합으로 관리디는 데이터 덩어리임.
# 기존에 있으면 삭제후 생성함
def make_index(es, index_name):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(es.indices.create(index=index_name))
    else:
        print(es.indices.create(index=index_name))
        
index_name = 'goods'
make_index(es, index_name)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'goods'}


In [4]:
# 데이터들 준비
doc1 = {'goods_name': '삼성 노트북 9', 'price': 100000}
doc2 = {'goods_name': '엘지 노트북 그램', 'price': 200000}
doc3 = {'goods_name': '애플 맥북 프로',   'price': 3000000}

In [5]:
# 인덱스에 데이터 추가
es.index(index=index_name, doc_type='string', body=doc1)
es.index(index=index_name, doc_type='string', body=doc2)
es.index(index=index_name, doc_type='string', body=doc3)

# 인덱스 refresh 함
# elasticsearch의 자동 새로고침의 시간은 1초 정도 소요
# 따라서 코드에 아래 명령어를 입력하지 않았을 경우 검색을 하지 못할 가능성도 존재
    
es.indices.refresh(index=index_name)

C:\Users\bong9\anaconda3\envs\pytorch\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


{'_shards': {'total': 2, 'successful': 2, 'failed': 0}}

In [7]:
# 검색 해 봄  
results = es.search(index=index_name, body={'from':0, 'size':10, 'query':{'match':{'goods_name':'노트북'}}})
for result in results['hits']['hits']:
    print('score:', result['_score'], 'source:', result['_source'])

score: 0.4700036 source: {'goods_name': '삼성 노트북 9', 'price': 100000}
score: 0.4700036 source: {'goods_name': '엘지 노트북 그램', 'price': 200000}
